In [1]:
pip install psycopg2


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import psycopg2

In [3]:
#Creamos una variable para crear la conexión.
conn = psycopg2.connect(
    dbname = "six_majors",
    user = "my_user",
    password = "admin",
    host = "localhost",
    port = "5432"
)

In [4]:
cur = conn.cursor()

In [5]:
cur.execute("SELECT version();")
cur.fetchone()

('PostgreSQL 14.17 (Homebrew) on aarch64-apple-darwin24.2.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit',)

In [7]:
df_six_majors = pd.read_excel("../data/datos_limpios/resultados_six_majors_2024.xlsx")
df_six_majors.head()

,surname,name,country,age_group,finish_time,gender,place,place_gender,marathon
0,Mengesha,Milkesa,ETH,H,02:03:17,M,1,1,Berlin Marathon
1,Kotut,Cybrian,KEN,30,02:03:22,M,2,2,Berlin Marathon
2,Alew,Haymanot,ETH,H,02:03:31,M,3,3,Berlin Marathon
3,Kiprop,Stephen,KEN,H,02:03:37,M,4,4,Berlin Marathon
4,Kiros,Hailemariyam,ETH,H,02:04:35,M,5,5,Berlin Marathon


In [8]:
#Para preparar la primera carga, creamos una tabla llamada df_marathons con los valores únicos de la columna marathon:
df_marathons = df_six_majors[['marathon']].drop_duplicates()
df_marathons

,marathon
0,Berlin Marathon
54096,Boston Marathon
79598,Chicago Marathon
131462,London Marathon


In [9]:
#Ahora creamos una columna en df_marathons llamada id_marathon, que será la clave primaria de la tabla:
df_marathons['id_marathon'] = np.arange(1, len(df_marathons) + 1)
df_marathons

,marathon,id_marathon
0,Berlin Marathon,1
54096,Boston Marathon,2
79598,Chicago Marathon,3
131462,London Marathon,4


In [10]:
insert_query = """

        INSERT INTO marathons (id_marathon, marathon)
        VALUES (%s, %s)    
"""

In [11]:
data_to_insert = []
for index, row in df_marathons.iterrows():
    data_to_insert.append((row['id_marathon'], row['marathon']))
#Insertamos los datos en la tabla marathons:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [13]:
#comprobamos que se han insertado los datos correctamente:
query_com = """
        SELECT * FROM marathons
"""
cur.execute(query_com)
cur.fetchall()

[(1, 'Berlin Marathon'),
 (2, 'Boston Marathon'),
 (3, 'Chicago Marathon'),
 (4, 'London Marathon')]

In [15]:
#Para la tabla country, creamos una tabla llamada df_country con los valores únicos de la columna country:
df_country = df_six_majors[['country']].drop_duplicates()
df_country
#Ahora creamos una columna en df_country llamada id_country, que será la clave primaria de la tabla:
df_country['id_country'] = np.arange(1, len(df_country) + 1)
df_country.head()

,country,id_country
0,ETH,1
1,KEN,2
5,JPN,3
7,ERI,4
15,ISR,5


In [16]:
insert_query = """
INSERT INTO country (id_country, country)
VALUES (%s, %s)    
"""
data_to_insert = []
for index, row in df_country.iterrows():
    data_to_insert.append((row['id_country'], row['country']))
#Insertamos los datos en la tabla country:
cur.executemany(insert_query, data_to_insert)
conn.commit()
#comprobamos que se han insertado los datos correctamente:
query_com = """
        SELECT * FROM country
"""
cur.execute(query_com)
cur.fetchall()

[(1, 'ETH'),
 (2, 'KEN'),
 (3, 'JPN'),
 (4, 'ERI'),
 (5, 'ISR'),
 (6, 'GER'),
 (7, 'DEN'),
 (8, 'CHN'),
 (9, 'AUT'),
 (10, 'NED'),
 (11, 'USA'),
 (12, 'SWE'),
 (13, 'GRE'),
 (14, 'FRA'),
 (15, 'GBR'),
 (16, 'MGL'),
 (17, 'KAZ'),
 (18, 'TPE'),
 (19, 'GUA'),
 (20, 'ISL'),
 (21, 'BEL'),
 (22, 'BUL'),
 (23, 'ITA'),
 (24, 'ARG'),
 (25, 'VIE'),
 (26, 'NOR'),
 (27, 'ESP'),
 (28, 'MEX'),
 (29, 'SUI'),
 (30, 'IRL'),
 (31, 'RUS'),
 (32, 'AUS'),
 (33, 'FIN'),
 (34, 'POL'),
 (35, 'COL'),
 (36, 'CZE'),
 (37, 'CHI'),
 (38, 'CRC'),
 (39, 'BIH'),
 (40, 'NZL'),
 (41, 'HUN'),
 (42, 'LAT'),
 (43, 'DOM'),
 (44, 'BRA'),
 (45, 'ROU'),
 (46, 'PER'),
 (47, 'RSA'),
 (48, 'MDA'),
 (49, 'POR'),
 (50, 'EST'),
 (51, 'LTU'),
 (52, 'SVK'),
 (53, 'CYP'),
 (54, 'CAN'),
 (55, 'ECU'),
 (56, 'PUR'),
 (57, 'TUR'),
 (58, 'KOR'),
 (59, 'HKG'),
 (60, 'SLO'),
 (61, 'SGP'),
 (62, 'SRB'),
 (63, 'BLR'),
 (64, 'AFG'),
 (65, 'UKR'),
 (66, 'BUR'),
 (67, 'IND'),
 (68, 'IRI'),
 (69, 'VEN'),
 (70, 'THA'),
 (71, 'LBN'),
 (72, 'QAT'),
 

In [17]:
df_sponsors = pd.read_excel("../data/datos_limpios/sponsors.xlsx")
df_sponsors.head()

,sponsor,marathon
0,BMW,Berlin Marathon
1,adidas,Berlin Marathon
2,Abbott,Berlin Marathon
3,Zalando,Berlin Marathon
4,TCS,London Marathon


In [18]:
#asignamos la columna id_marathon a la tabla df_sponsors para repartir cada sponsor con su respectivo marathon:
df_sponsors['id_marathon'] = df_sponsors['marathon'].map(df_marathons.set_index('marathon')['id_marathon'])
df_sponsors.head()


,sponsor,marathon,id_marathon
0,BMW,Berlin Marathon,1
1,adidas,Berlin Marathon,1
2,Abbott,Berlin Marathon,1
3,Zalando,Berlin Marathon,1
4,TCS,London Marathon,4


In [19]:
insert_query = """
INSERT INTO sponsors (id_marathon, sponsor)
VALUES (%s, %s)    
"""
data_to_insert = []
for index, row in df_sponsors.iterrows():
    data_to_insert.append((row['id_marathon'], row['sponsor']))
#Insertamos los datos en la tabla sponsors:
cur.executemany(insert_query, data_to_insert)
conn.commit()
#comprobamos que se han insertado los datos correctamente:
query_com = """
        SELECT * FROM sponsors
"""
cur.execute(query_com)
cur.fetchall()

[(1, 1, 'BMW'),
 (2, 1, 'adidas'),
 (3, 1, 'Abbott'),
 (4, 1, 'Zalando'),
 (5, 4, 'TCS'),
 (6, 4, 'New Balance'),
 (7, 4, 'Abbott'),
 (8, 4, 'Buxton'),
 (9, 4, 'Coopah'),
 (10, 4, 'enthuse'),
 (11, 4, 'Ford'),
 (12, 4, 'GetPRO'),
 (13, 4, 'HubSpot'),
 (14, 4, 'iFIT'),
 (15, 4, 'Lucozade'),
 (16, 4, 'Marathon Tour & Travel'),
 (17, 4, 'Radox'),
 (18, 4, 'Shokz'),
 (19, 4, 'Sports Tours International'),
 (20, 4, 'Tag Heuer'),
 (21, 4, 'Voltarol'),
 (22, 3, 'Bank of America'),
 (23, 3, 'Nike'),
 (24, 3, 'Abbott'),
 (25, 3, 'TCS'),
 (26, 3, 'Advocate Health Care'),
 (27, 3, 'KIA'),
 (28, 3, 'Gatorade'),
 (29, 3, 'Biofreeze'),
 (30, 2, 'Bank of America'),
 (31, 2, 'adidas'),
 (32, 2, 'Schneider Electric'),
 (33, 2, 'ESPN'),
 (34, 2, 'TCS'),
 (35, 2, 'Gatorade'),
 (36, 2, 'Samuel Adams'),
 (37, 2, 'Abbott'),
 (38, 2, 'Honda'),
 (39, 2, 'Dicks')]

In [22]:
#asignamos a la tabla df_six_majors las columnas id_marathon y id_country:
df_six_majors['id_marathon'] = df_six_majors['marathon'].map(df_marathons.set_index('marathon')['id_marathon'])
df_six_majors['id_country'] = df_six_majors['country'].map(df_country.set_index('country')['id_country'])
df_six_majors.sample(10)

,surname,name,country,age_group,finish_time,gender,place,place_gender,marathon,id_marathon,id_country
5928,Beltrami,Luca,ITA,45,03:07:42,M,5929,5323,Berlin Marathon,1,23
13795,FRANCESCHI,SABRINA,FRA,40,03:31:54,W,13799,2258,Berlin Marathon,1,14
24706,Nevers,Maggie,USA,30,03:57:20,W,24713,5535,Berlin Marathon,1,11
142339,Sara,Finch,AUS,55-59,03:32:06,W,10878,2243,London Marathon,4,32
153753,Tom,Hunt,GBR,18-39,04:04:14,M,22292,15949,London Marathon,4,15
87255,Michael,Faraj,USA,40-44,03:18:16,M,7658,6512,Chicago Marathon,3,11
126328,Rebecca,Prata,USA,25-29,05:44:38,W,46731,20510,Chicago Marathon,3,11
3874,Bui,Anh,USA,30,02:58:07,W,3875,277,Berlin Marathon,1,11
179372,Julie,Feeney,GBR,55-59,05:51:10,W,47911,18936,London Marathon,4,15
115310,Lauren,Butler,USA,25-29,04:44:15,W,35713,14046,Chicago Marathon,3,11


In [23]:
#sacamos los valores únicos de la coluna age_group:
df_age_group = df_six_majors[['age_group']].drop_duplicates()
df_age_group

,age_group
0,H
1,30
19,JU20
33,35
63,40
114,55
122,50
129,45
1978,60
3746,65


In [25]:
#Duplicamos el df_six_majors con un df llamado df_final:
df_final = df_six_majors.copy()
df_final.head()

,surname,name,country,age_group,finish_time,gender,place,place_gender,marathon,id_marathon,id_country
0,Mengesha,Milkesa,ETH,H,02:03:17,M,1,1,Berlin Marathon,1,1
1,Kotut,Cybrian,KEN,30,02:03:22,M,2,2,Berlin Marathon,1,2
2,Alew,Haymanot,ETH,H,02:03:31,M,3,3,Berlin Marathon,1,1
3,Kiprop,Stephen,KEN,H,02:03:37,M,4,4,Berlin Marathon,1,2
4,Kiros,Hailemariyam,ETH,H,02:04:35,M,5,5,Berlin Marathon,1,1


In [37]:
#Sacamos los valores únicos de la columna age_group:
df_age_group = df_six_majors[['age_group']].drop_duplicates()
df_age_group

,age_group
0,H
1,30
19,JU20
33,35
63,40
114,55
122,50
129,45
1978,60
3746,65


In [ ]:
#quiero ver las filas que tienen el valor H de la columna age_group:
df_final[df_final['age_group'] == '']

,surname,name,country,age_group,finish_time,gender,place,place_gender,marathon,id_marathon,id_country,grupo_edad_unificado


In [ ]:
#para poder unificar los valores de la columna age_group:
def unificar_grupo_edad(grupo):
    if pd.isna(grupo):
        return None
    grupo = str(grupo).strip().lower()
    if grupo in ["ju20", "19 and under"]:
        return "JU20"
    elif grupo in ["20-24"]:
        return "20-24"
    elif grupo in ["25-29"]:
        return "25-29"
    elif grupo == "18-39":
        return "30-39"
    elif grupo in ["30", "30-34"]:
        return "30-34"
    elif grupo in ["35", "35-39"]:
        return "35-39"
    elif grupo in ["40", "40-44"]:
        return "40-44"
    elif grupo in ["45", "45-49"]:
        return "45-49"
    elif grupo in ["50", "50-54"]:
        return "50-54"
    elif grupo in ["55", "55-59"]:
        return "55-59"
    elif grupo in ["60", "60-64"]:
        return "60-64"
    elif grupo in ["65", "65-69"]:
        return "65-69"
    elif grupo in ["70", "70-74"]:
        return "70-74"
    elif grupo in ["75", "75-79"]:
        return "75-79"
    elif grupo in ["80", "80+"]:
        return "80+"
    elif grupo == "18-39":
        return "30-39"  
    elif grupo == "H":
        return "-"
    else:
        return "-"

# Aplicación sobre el DataFrame
df_final["grupo_edad_unificado"] = df_final["age_group"].apply(unificar_grupo_edad)

In [ ]:
elif grupo == "18-39":
    return "30-39"

In [47]:
df_final.head(
)

,surname,name,country,finish_time,gender,place,place_gender,marathon,id_marathon,id_country,age_group,grupo_edad_unificado
0,Mengesha,Milkesa,ETH,02:03:17,M,1,1,Berlin Marathon,1,1,-,-
1,Kotut,Cybrian,KEN,02:03:22,M,2,2,Berlin Marathon,1,2,30-34,30-34
2,Alew,Haymanot,ETH,02:03:31,M,3,3,Berlin Marathon,1,1,-,-
3,Kiprop,Stephen,KEN,02:03:37,M,4,4,Berlin Marathon,1,2,-,-
4,Kiros,Hailemariyam,ETH,02:04:35,M,5,5,Berlin Marathon,1,1,-,-


In [48]:
#vemos los valores únicos de la columna grupo_edad_unificado:
df_final['grupo_edad_unificado'].unique()

array(['-', '30-34', 'JU20', '35-39', '40-44', '55-59', '50-54', '45-49',
       '60-64', '65-69', '70-74', '75-79', '80+', '25-29', '20-24'],
      dtype=object)

In [49]:
#eliminamos la columna age_group y renombramos la columna grupo_edad_unificado a age_group:
df_final.drop(columns=['age_group'], inplace=True)
df_final.rename(columns={'grupo_edad_unificado': 'age_group'}, inplace=True)
df_final.head()

,surname,name,country,finish_time,gender,place,place_gender,marathon,id_marathon,id_country,age_group
0,Mengesha,Milkesa,ETH,02:03:17,M,1,1,Berlin Marathon,1,1,-
1,Kotut,Cybrian,KEN,02:03:22,M,2,2,Berlin Marathon,1,2,30-34
2,Alew,Haymanot,ETH,02:03:31,M,3,3,Berlin Marathon,1,1,-
3,Kiprop,Stephen,KEN,02:03:37,M,4,4,Berlin Marathon,1,2,-
4,Kiros,Hailemariyam,ETH,02:04:35,M,5,5,Berlin Marathon,1,1,-


In [51]:
#Creamos una tabla con los valores únicos de la columna age_group:
df_age_group = df_final[['age_group']].drop_duplicates()
df_age_group
#Ahora creamos una columna en df_age_group llamada id_age_group, que será la clave primaria de la tabla:
df_age_group['id_age_group'] = np.arange(1, len(df_age_group) + 1)
df_age_group

,age_group,id_age_group
0,-,1
1,30-34,2
19,JU20,3
33,35-39,4
63,40-44,5
114,55-59,6
122,50-54,7
129,45-49,8
1978,60-64,9
3746,65-69,10


In [54]:
insert_query = """
INSERT INTO age (id_age_group, age_group)
VALUES (%s, %s)    
"""
data_to_insert = []
for index, row in df_age_group.iterrows():
    data_to_insert.append((row['id_age_group'], row['age_group']))
#Insertamos los datos en la tabla age:
cur.executemany(insert_query, data_to_insert)
conn.commit()
#comprobamos que se han insertado los datos correctamente:
query_com = """
        SELECT * FROM age
"""
cur.execute(query_com)
cur.fetchall()


UniqueViolation: duplicate key value violates unique constraint "age_pkey"
DETAIL:  Key (id_age_group)=(1) already exists.


In [57]:
#borramos los datos cargados a la tabla age para volver a cargarlos corregidos:
delete_query = """
DELETE FROM age
"""
cur.execute(delete_query)
conn.commit()

In [58]:
#volvemos a cargar los datos corregidos:
insert_query = """
INSERT INTO age (id_age_group, age_group)
VALUES (%s, %s)    
"""
data_to_insert = []
for index, row in df_age_group.iterrows():
    data_to_insert.append((row['id_age_group'], row['age_group']))
#Insertamos los datos en la tabla age:
cur.executemany(insert_query, data_to_insert)
conn.commit()
#comprobamos que se han insertado los datos correctamente:
query_com = """
        SELECT * FROM age
"""
cur.execute(query_com)
cur.fetchall()

[(1, '-'),
 (2, '30-34'),
 (3, 'JU20'),
 (4, '35-39'),
 (5, '40-44'),
 (6, '55-59'),
 (7, '50-54'),
 (8, '45-49'),
 (9, '60-64'),
 (10, '65-69'),
 (11, '70-74'),
 (12, '75-79'),
 (13, '80+'),
 (14, '25-29'),
 (15, '20-24')]

In [56]:
conn.rollback()

CREATE TABLE six_majors (
    id SERIAL PRIMARY KEY,
    place INT,
    place_gender INT,
    name VARCHAR(100),
    surname VARCHAR(100),
    gender VARCHAR(1), -- M o W
    id_country INT REFERENCES country(id_country),
    id_age_group INT REFERENCES age(id_age_group),
    id_marathon INT REFERENCES marathons(id_marathon),
    finish_time TIME
);

In [59]:
#duplicamos el df_final con df_six_majors:
df_six_majors = df_final.copy()
df_six_majors.head()

,surname,name,country,finish_time,gender,place,place_gender,marathon,id_marathon,id_country,age_group
0,Mengesha,Milkesa,ETH,02:03:17,M,1,1,Berlin Marathon,1,1,-
1,Kotut,Cybrian,KEN,02:03:22,M,2,2,Berlin Marathon,1,2,30-34
2,Alew,Haymanot,ETH,02:03:31,M,3,3,Berlin Marathon,1,1,-
3,Kiprop,Stephen,KEN,02:03:37,M,4,4,Berlin Marathon,1,2,-
4,Kiros,Hailemariyam,ETH,02:04:35,M,5,5,Berlin Marathon,1,1,-


In [60]:
#añadimos las columnas id a df_six_majors:
df_six_majors['id_country'] = df_six_majors['country'].map(df_country.set_index('country')['id_country'])
df_six_majors['id_marathon'] = df_six_majors['marathon'].map(df_marathons.set_index('marathon')['id_marathon'])
df_six_majors['id_age_group'] = df_six_majors['age_group'].map(df_age_group.set_index('age_group')['id_age_group'])
df_six_majors.head()

,surname,name,country,finish_time,gender,place,place_gender,marathon,id_marathon,id_country,age_group,id_age_group
0,Mengesha,Milkesa,ETH,02:03:17,M,1,1,Berlin Marathon,1,1,-,1
1,Kotut,Cybrian,KEN,02:03:22,M,2,2,Berlin Marathon,1,2,30-34,2
2,Alew,Haymanot,ETH,02:03:31,M,3,3,Berlin Marathon,1,1,-,1
3,Kiprop,Stephen,KEN,02:03:37,M,4,4,Berlin Marathon,1,2,-,1
4,Kiros,Hailemariyam,ETH,02:04:35,M,5,5,Berlin Marathon,1,1,-,1


CREATE TABLE six_majors (
    id SERIAL PRIMARY KEY,
    place INT,
    place_gender INT,
    name VARCHAR(100),
    surname VARCHAR(100),
    gender VARCHAR(1), -- M o W
    id_country INT REFERENCES country(id_country),
    id_age_group INT REFERENCES age(id_age_group),
    id_marathon INT REFERENCES marathons(id_marathon),
    finish_time TIME
);

In [66]:
insert_query = """
INSERT INTO six_majors (place, place_gender, name, surname, gender, id_country, id_age_group, id_marathon, finish_time)
VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s)    
"""
data_to_insert = []
for index, row in df_six_majors.iterrows():
    data_to_insert.append((row['place'], row['place_gender'], row['name'], row['surname'],row['gender'], row['id_country'], row['id_age_group'], row['id_marathon'], row['finish_time']))
#Insertamos los datos en la tabla six_majors:
cur.executemany(insert_query, data_to_insert)
conn.commit()
#comprobamos que se han insertado los datos correctamente:
query_com = """
        SELECT * FROM six_majors
"""
cur.execute(query_com)
cur.fetchall()

[(1, 1, 1, ' Milkesa', 'Mengesha', 'M', 1, 1, 1, datetime.time(2, 3, 17)),
 (2, 2, 2, ' Cybrian', 'Kotut', 'M', 2, 2, 1, datetime.time(2, 3, 22)),
 (3, 3, 3, ' Haymanot', 'Alew', 'M', 1, 1, 1, datetime.time(2, 3, 31)),
 (4, 4, 4, ' Stephen', 'Kiprop', 'M', 2, 1, 1, datetime.time(2, 3, 37)),
 (5, 5, 5, ' Hailemariyam', 'Kiros', 'M', 1, 1, 1, datetime.time(2, 4, 35)),
 (6, 6, 6, ' Yohei', 'Ikeda', 'M', 3, 1, 1, datetime.time(2, 5, 12)),
 (7, 7, 7, ' Tadese', 'Takele', 'M', 1, 1, 1, datetime.time(2, 5, 13)),
 (8, 8, 8, ' Oqbe Kibrom', 'Ruesom', 'M', 4, 1, 1, datetime.time(2, 5, 37)),
 (9, 9, 9, ' Enock', 'Onchari', 'M', 2, 1, 1, datetime.time(2, 5, 53)),
 (10,
  10,
  10,
  ' DERSEH KINDIE',
  'KASSIE',
  'M',
  1,
  1,
  1,
  datetime.time(2, 5, 54)),
 (11, 11, 11, ' Melaku', 'Belachew', 'M', 1, 2, 1, datetime.time(2, 6, 30)),
 (12, 12, 12, ' Luke', 'Kiprop', 'M', 2, 1, 1, datetime.time(2, 6, 39)),
 (13, 13, 13, ' Kibiwott', 'Kandie', 'M', 2, 1, 1, datetime.time(2, 6, 46)),
 (14, 14, 14,

In [65]:
#hacemos rollback para deshacer la transacción:
conn.rollback()